<a href="https://colab.research.google.com/github/Ramjeet-Dixit/IITM-AIML-Rdixit/blob/main/text_classification_rnn_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Recurrent Neural Networks**

I love you



i you love

hey, i am nikhil and I am here to ....... on neural networks.

time series like stock price

# Text classification with an RNN

The Long Short-Term Memory network, or LSTM network, is a recurrent neural network that is trained using Backpropagation Through Time

As such, it can be used to create large recurrent networks that in turn can be used to address difficult sequence problems in machine learning and achieve state-of-the-art results.

Instead of neurons, LSTM networks have memory blocks that are connected through layers.

A block has components that make it smarter than a classical neuron and a memory for recent sequences. A block contains gates that manage the block’s state and output. A block operates upon an input sequence and each gate within a block uses the sigmoid activation units to control whether they are triggered or not, making the change of state and addition of information flowing through the block conditional.

There are three types of gates within a unit:

Forget Gate: conditionally decides what information to throw away from the block.

Input Gate: conditionally decides which values from the input to update the memory state.

Output Gate: conditionally decides what to output based on input and the memory of the block.

Each unit is like a mini-state machine where the gates of the units have weights that are learned during the training procedure.

You can see how you may achieve sophisticated learning and memory from a layer of LSTMs, and it is not hard to imagine how higher-order abstractions may be layered with multiple such layers.

## Setup

In [ ]:
import numpy as np

import tensorflow_datasets as tfds
#used for loading the imdb reviews data
import tensorflow as tf

## Setup input pipeline


The IMDB large movie review dataset is a *binary classification* dataset—all the reviews have either a *positive* or *negative* sentiment.

Download the dataset using [TFDS](https://www.tensorflow.org/datasets). See the [loading text tutorial](https://www.tensorflow.org/tutorials/load_data/text) for details on how to load this sort of data manually.


In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.TX3DJ5_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.TX3DJ5_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.TX3DJ5_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
#print out the keys of the dataset dictionary
dataset.keys()

dict_keys([Split('train'), Split('test'), Split('unsupervised')])

In [ ]:
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
type(train_dataset)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [ ]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


Some common arguments:

split=: Which split to read (e.g. 'train', ['train', 'test'], 'train[80%:]',...). See our split API guide.

shuffle_files=: Control whether to shuffle the files between each epoch (TFDS store big datasets in multiple smaller files).

data_dir=: Location where the dataset is saved ( defaults to ~/tensorflow_datasets/)

with_info=True: Returns the tfds.core.DatasetInfo containing dataset metadata

download=False: Disable download

Initially this returns a dataset of (text, label pairs):

Next shuffle the data for training and create batches of these `(text, label)` pairs:

In [ ]:
BUFFER_SIZE = 10000 #virtual pipeline to put input in ready to go
BATCH_SIZE = 64

buffer_size: A tf.int64 scalar tf.Tensor, representing the maximum number elements that will be buffered when prefetching.

Shuffling begins by making a buffer of size BUFFER_SIZE (which starts empty but has enough room to store that many elements).

The buffer is then filled until it has no more capacity with elements from the dataset, then an element is chosen uniformly at random.

This means that each example in the buffer is equally likely to be chosen, with probability 1/BUFFER_SIZE.

Then, a new example is loaded to fill the slot in the buffer that was emptied. This continues until there is nothing left to load.

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

The tf.data API provides the tf.data.Dataset.prefetch transformation. It can be used to decouple the time when data is produced from the time when data is consumed.

In particular, the transformation uses a background thread and an internal buffer to prefetch elements from the input dataset ahead of the time they are requested.

The number of elements to prefetch should be equal to (or possibly greater than) the number of batches consumed by a single training step. You could either manually tune this value, or set it to tf.data.AUTOTUNE, which will prompt the tf.data runtime to tune the value dynamically at runtime.


while the training step is running for sample 0, the input pipeline is reading the data for the sample 1, and so on.

In [ ]:
for example, label in train_dataset.take(1):#1 batch = 64 reviews
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b"This film is famous for several qualities: a literate script, for once in partly-religious film-making, by Philip Dunne, some very good performances, a first-rate production in every department and its intelligent direction by veteran Henry King. If one were making a film, then getting such talents as Leon Shamroy as cinematographer, Lyle Wheeler as art director and Alfred Newman as composer of original music would guarantee a quality production. Add the cast of this film, including Gregory Peck and Susan Hayward as the title characters, James Robertson Justice, Raymond Massey, Kieron Moore, Jayne Meadows and John Sutton plus a dance by Gwen Verdon and expectations might be raised that the resulting film could be made into something special. But in a biblical subject script, usually a sub-genre prone to illogical motivations and miraculous interventions, everything would ultimately depend on the author's skills. Philip Dunne here has supplied human beings, a rare achievement

In [ ]:
info.features

FeaturesDict({
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    'text': Text(shape=(), dtype=string),
})



```
`# This is formatted as code`
```

## Create the text encoder

The raw text loaded by `tfds` needs to be processed before it can be used in a model. The simplest way to process text for training is using the `TextVectorization` layer. This layer has many capabilities, but this tutorial sticks to the default behavior.

Create the layer, and pass the dataset's text to the layer's `.adapt` method:

This layer has basic options for managing text in a Keras model. It transforms a batch of strings (one sample = one string) into either a list of token indices (one sample = 1D tensor of integer token indices) or a dense representation (one sample = 1D tensor of float values representing data about the sample's tokens).

If desired, the user can call this layer's adapt() method on a dataset. When this layer is adapted, it will analyze the dataset, determine the frequency of individual string values, and create a 'vocabulary' from them. This vocabulary can have unlimited size or be capped, depending on the configuration options for this layer; if there are more unique values in the input than the maximum vocabulary size, the most frequent terms will be used to create the vocabulary.

The processing of each sample contains the following steps:

standardize each sample (usually lowercasing + punctuation stripping)

split each sample into substrings (usually words)

recombine substrings into tokens (usually ngrams)

index tokens (associate a unique int value with each token (unique word))

transform each sample using this index, either into a vector of ints or a dense float vector.

In [ ]:
VOCAB_SIZE = 1000 #dictionary of unique words in a document
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))


| Configuration                                         | What happens                               |
| ----------------------------------------------------- | ------------------------------------------ |
| `standardize="lower_and_strip_punctuation"` (default) | **Removes @ ? ! # $ %**, etc.              |
| `standardize=None`                                    | Keeps all characters                       |
| Custom function                                       | You choose which characters to keep/remove |


The `.adapt` method sets the layer's vocabulary. Here are the first 20 tokens. After the padding and unknown tokens they're sorted by frequency:

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

Once the vocabulary is set, the layer can encode text into indices. The tensors of indices are 0-padded to the longest sequence in the batch (unless you set a fixed `output_sequence_length`):

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[11, 20,  7, ...,  0,  0,  0],
       [11,  7, 29, ...,  0,  0,  0],
       [49, 85,  1, ...,  0,  0,  0]])

With the default settings, the process is not completely reversible. There are three main reasons for that:

1. The default value for `preprocessing.TextVectorization`'s `standardize` argument is `"lower_and_strip_punctuation"`.
2. The limited vocabulary size and lack of character-based fallback results in some unknown tokens.

'lower_and_strip_punctuation' (lowercase and remove punctuation) .Default is 'lower_and_strip_punctuation'.

In [ ]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b"This film is famous for several qualities: a literate script, for once in partly-religious film-making, by Philip Dunne, some very good performances, a first-rate production in every department and its intelligent direction by veteran Henry King. If one were making a film, then getting such talents as Leon Shamroy as cinematographer, Lyle Wheeler as art director and Alfred Newman as composer of original music would guarantee a quality production. Add the cast of this film, including Gregory Peck and Susan Hayward as the title characters, James Robertson Justice, Raymond Massey, Kieron Moore, Jayne Meadows and John Sutton plus a dance by Gwen Verdon and expectations might be raised that the resulting film could be made into something special. But in a biblical subject script, usually a sub-genre prone to illogical motivations and miraculous interventions, everything would ultimately depend on the author's skills. Philip Dunne here has supplied human beings, a rare achieveme

## Create the model

![A drawing of the information flow in the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/bidirectional.png?raw=1)

Above is a diagram of the model.

1. This model can be build as a `tf.keras.Sequential`.

2. The first layer is the `encoder`, which converts the text to a sequence of token indices.

3. After the encoder is an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

Since the large vocabulary size will make the manipulations complicated; it should be embedded into a small-sized vector representation

  This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a `tf.keras.layers.Dense` layer.

4. A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input on the next timestep.

  The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the final output.

  * The main advantage of a bidirectional RNN is that the signal from the beginning of the input doesn't need to be processed all the way through every timestep to affect the output.  

  * The main disadvantage of a bidirectional RNN is that you can't efficiently stream predictions as words are being added to the end.

5. After the RNN has converted the sequence to a single vector the two `layers.Dense` do some final processing, and convert from this vector representation to a single logit as the classification output.


The code to implement this is below:

"I love pizza" → [12, 87, 34]

[12, 87, 34] to the embedding layer

Each token becomes a 64-dimensional vector:

[
  embedding_of_12 (shape 64),
  embedding_of_87 (shape 64),
  embedding_of_34 (shape 64)
]

(3, 64) -- FInal Shape

[
  [0.12, -0.07, ... 64 dims ...],
  [-0.33, 0.55, ... 64 dims ...],
  [0.92, 0.14, ... 64 dims ...]
]

This sequence now contains learned meaning, not arbitrary integers.



| Vocabulary Size | Recommended Embedding Dim |
| --------------- | ------------------------- |
| < 5,000         | 16–64                     |
| 5,000–20,000    | 64–128                    |
| > 20,000        | 128–300                   |
| > 200,000       | 300–600 (rarely needed)   |


| Task                              | Suggested Dim |
| --------------------------------- | ------------- |
| Simple text classification        | 32–64         |
| Sentiment analysis                | 50–128        |
| Machine translation               | 256–512       |
| Question answering / advanced NLP | 300–768+      |


Training time + memory constraints

Larger embedding → more parameters → slower training.

Number of parameters = vocab_size * embedding_dim

Example:

If vocab = 20,000
Embedding 64 → 1.28M params
Embedding 256 → 5.12M params

So bigger isn’t always better.

embedding_dim = min(300, round(6 * vocab_size^(1/4)))


In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,#nodes
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),#RNN layer
    tf.keras.layers.Dense(64, activation='relu'),#hidden or fullyconnected layer
    tf.keras.layers.Dense(1)#output layer
])

In [ ]:
#xase 2: with 2 bidirectional layers and dropout
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (64, None)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Please note that Keras sequential model is used here since all the layers in the model only have single input and produce single output. In case you want to use stateful RNN layer, you might want to build your model with Keras functional API or model subclassing so that you can retrieve and reuse the RNN layer states. Please check [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) for more details.

The embedding layer [uses masking](https://www.tensorflow.org/guide/keras/masking_and_padding) to handle the varying sequence-lengths. All the layers after the `Embedding` support masking:

Masking is a way to tell sequence-processing layers that certain timesteps in an input are missing, and thus should be skipped when processing the data.

Padding is a special form of masking where the masked steps are at the start or the end of a sequence. Padding comes from the need to encode sequence data into contiguous batches: in order to make all sequences in a batch fit a given standard length, it is necessary to pad or truncate some sequences.

Compile the Keras model to configure the training process:

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

## Train the model

In [ ]:
new = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 29s 63ms/step - accuracy: 0.5161 - loss: 0.6815 - val_accuracy: 0.7516 - val_loss: 0.4753
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 25s 63ms/step - accuracy: 0.7612 - loss: 0.4886 - val_accuracy: 0.8302 - val_loss: 0.4039
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.8288 - loss: 0.3818 - val_accuracy: 0.8141 - val_loss: 0.3758
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 25s 61ms/step - accuracy: 0.8459 - loss: 0.3500 - val_accuracy: 0.8438 - val_loss: 0.3484
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 60ms/step - accuracy: 0.8543 - loss: 0.3339 - val_accuracy: 0.8484 - val_loss: 0.3360
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 60ms/step - accuracy: 0.8646 - loss: 0.3163 - val_accuracy: 0.8536 - val_loss: 0.3322
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 60ms/step - accuracy: 0.8644 - loss: 0.3131 - val_accuracy: 0.8542 - val_loss: 0.3312
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.8697 - loss: 0.3059 - 

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

391/391 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.8635 - loss: 0.3255
Test Loss: 0.3239615559577942
Test Accuracy: 0.8625199794769287


In [ ]:
train_loss, train_acc = model.evaluate(train_dataset)

print('Test Loss:', train_loss)
print('Test Accuracy:', train_acc)

391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.8766 - loss: 0.3007
Test Loss: 0.29813387989997864
Test Accuracy: 0.880079984664917


Run a prediction on a new sentence:

If the prediction is >= 0.0, it is positive else it is negative.

In [ ]:
import tensorflow as tf
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(tf.constant([sample_text], dtype=tf.string))
print(predictions[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
[0.5944768]


In [ ]:
#convert above result into probability
import numpy as np

logit = 0.5944768
prob = 1 / (1 + np.exp(-logit))
print(prob)

0.6443916709387254


In [ ]:
model.summary()

## Stack two or more LSTM layers

Keras recurrent layers have two available modes that are controlled by the `return_sequences` constructor argument:

* If `False` it returns only the last output for each input sequence (a 2D tensor of shape (batch_size, output_features)). This is the default, used in the previous model.

* If `True` the full sequences of successive outputs for each timestep is returned (a 3D tensor of shape `(batch_size, timesteps, output_features)`).

Here is what the flow of information looks like with `return_sequences=True`:

![layered_bidirectional](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/layered_bidirectional.png?raw=1)

The interesting thing about using an `RNN` with `return_sequences=True` is that the output still has 3-axes, like the input, so it can be passed to another RNN layer, like this:

We feed in a sequence of inputs (x's), one batch at a time and each LSTM cell returns an output (y_i). So if your input is of size batch_size x time_steps X input_size then the LSTM output will be batch_size X time_steps X output_size.

This is called a sequence to sequence model because an input sequence is converted into an output sequence.

In keras this is achieved by setting return_sequences=True.

Sequence modeling, put simply, is the process of generating a sequence of values by analyzing a series of input value



In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True, recurrent_dropout=0.3)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

"I love deep learning"

LSTM processes step by step:

| Time Step | Word     | Hidden State |
| --------- | -------- | ------------ |
| t1        | I        | h1           |
| t2        | love     | h2           |
| t3        | deep     | h3           |
| t4        | learning | h4           |

while stacking RNN layers:

The second LSTM needs full sequence input.

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
train_loss, train_acc = model.evaluate(train_dataset)

print('Test Loss:', train_loss)
print('Test Accuracy:', train_acc)

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was not good. The animation and the graphics '
               'were terrible. I would not recommend this movie.')
predictions = model.predict((tf.constant([sample_text], dtype=tf.string)))
print(predictions)